In [2]:
from google.colab import drive 

drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
PATH = "/content/drive/MyDrive/CS678/AMBIPun/" # please change this by need
OPENAIKEY = "XXXX" # key included in report

In [4]:
!python3 -m pip install openai
!pip install simplet5
!pip install --upgrade simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd 
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import openai
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
import torch
import random

nltk.download('wordnet')

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def get_sense_key_pair(pun_word):
  df = pd.read_csv(PATH+"Dataset1.11.csv")
  idx = df[df['Pun']==pun_word].index.values[0]
  sense1 = df.iloc[idx]["Sense Key 1"]
  sense2 = df.iloc[idx]["Sense Key 2"] 
  return [sense1, sense2]

def get_sense_def_pair(pun_word):
  senseKeys= get_sense_key_pair(pun_word)
  sensePair = []
  for senseKey in senseKeys:
    m = wn.lemma_from_key(senseKey.split(";")[0]).synset().definition()
    sensePair.append(m)
  return sensePair

In [7]:
get_sense_def_pair("kid")

['be silly or tease one another', 'a young person of either sex']

In [8]:
openai.api_key = OPENAIKEY
model_engine = "text-davinci-003"

In [9]:
def format_response(response_text):
  response_text = response_text.split(". ")
  
  rel_words = []
  for r in response_text:
      word = r.split('\n')[0]
      if(word != ''):
          rel_words.append(word)
  return rel_words

def get_sense_rel_words(pun_word, n=3): 
  sense_def_pair = get_sense_def_pair(pun_word) 
  prompt1 = f"Generate {n} commonly used single words (except {pun_word}) that have meaning close to: {sense_def_pair[0]}" 
  prompt2 = f"Generate {n} commonly used single words (except {pun_word}) that have meaning close to: {sense_def_pair[1]}"

  response1 = openai.Completion.create(
        engine=model_engine,
        prompt=prompt1,
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.2,
    )

  response2 = openai.Completion.create(
        engine=model_engine,
        prompt=prompt2,
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.2,
    )

  response1 = response1.choices[0].text
  response2 = response2.choices[0].text
  
  result1 = format_response(response1)
  result2 = format_response(response2)

  return (result1, result2)

In [11]:
(rel_word1, rel_word2) = get_sense_rel_words("kid")
print(rel_word1)
print(rel_word2)

['Tease', 'Joke', 'Banter']
['Adolescent', 'Teenager', 'Youth']


In [12]:
def generateContextWords(related_word):
    openai.api_key = OPENAIKEY
    model_engine = "text-davinci-003"
    prompt = "Can you generate me seven relevant words to " + related_word + "?"
    completion = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.2,
    )
    response = completion.choices[0].text
    
    return format_response(response)

def generateAllContextWords(words):
    cw_list = []
    for w in words:
        cw = generateContextWords(w)
        for c in cw:
            cw_list.append(c)
    return cw_list

sense1Cw = generateAllContextWords(rel_word1)
sense2Cw = generateAllContextWords(rel_word2)

print(sense1Cw)
print(sense2Cw)


['Challenge', 'Entice', 'Provoke', 'Flirt', 'Taunt', 'Goad', 'Tease', 'Humor', 'Comedy', 'Punchline', 'Gag', 'Wit', 'Amusement', 'Jest', 'Wit', 'Repartee', 'Teasing', 'Joking', 'Humor', 'Jest', 'Quips']
['Puberty', 'Development', 'Maturity', 'Identity', 'Education', 'Socialization', 'Independence', 'School', 'Friends', 'Social Media', 'Music', 'Technology', 'Sports', 'Identity', 'Education', 'Engagement', 'Empowerment', 'Creativity', 'Opportunity', 'Leadership', 'Development']


In [104]:
# read in CSV file and name the columns
# df = pd.read_csv(PATH + 'dataset6.csv')

# # get context words 
# for idx, row in df.iterrows():
#   print(row['Pun'])
#   if row['Pun'] in ["sting", "pin", "parts", "power"]: 
#     continue
#   else: 
#     (rel_word1, rel_word2) = get_sense_rel_words(row['Pun'])
#     df.at[idx, 'Sense Key 1'] = generateAllContextWords(rel_word1) 
#     df.at[idx, 'Sense Key 2'] = generateAllContextWords(rel_word2)
# print(df.head(5))

trash
steps
cushion
Fair
fired
plastered
stick
buoyant
takeoff
well
spring
turns
ventured
feel
shot
hooked
bank
credit
position
fall
bounced
steps
instrumental
legends
high
chew
time
degrees
waste
battery
played
bright
lame
   Unnamed: 0           ID      Pun  \
0        1265  hom_2186_16    trash   
1        1266   hom_2187_4    steps   
2        1267  hom_2192_11  cushion   
3        1268   hom_2200_9     Fair   
4        1269  hom_2202_11    fired   

                                         Sense Key 1  \
0  [Reject, Deny, Decline, Rebut, Rejecting, Disa...   
1  [Gait, Pace, Step, Strut, March, Trot, Amble, ...   
2  [Protection, Defense, Security, Armor, Safegua...   
3  [Unbiased, Neutral, Disinterested, Unprejudice...   
4                                                 []   

                                         Sense Key 2  \
0  [Nonsense, Jargon, Babble, Drivel, Claptrap, B...   
1  [Staircase, Step, Ladder, Elevator, Platform, ...   
2                                   

In [13]:
#df.to_csv(PATH+"pun_with_context.csv")
context_df = pd.read_csv(PATH + "pun_with_context.csv")
context_df['Sense Key 1'] = context_df.apply(lambda row:  row['Sense Key 1'].strip('[]').replace("'", "").replace(' ', '').split(','), axis=1)
context_df['Sense Key 2'] = context_df.apply(lambda row:  row['Sense Key 2'].strip('[]').replace("'", "").replace(' ', '').split(','), axis=1)
context_df['Sense Key 1']

0       [Pressure, Engage, Grip, Transmission, Frictio...
1       [Rotate, Spin, Cycle, Orbit, Recur, Revolving,...
2       [Increase, Rise, Intensify, Expand, Inflate, A...
3                                                      []
4                                                      []
                              ...                        
1284                                              [waste]
1285                                            [battery]
1286                                             [played]
1287                                             [bright]
1288                                               [lame]
Name: Sense Key 1, Length: 1289, dtype: object

In [14]:
# create new dataframe with formatted columns
# new_df = pd.DataFrame({'source_text': 'generate sentence: ' + df['pun'] + ", " + df['sense1'].apply(lambda x: x.split('%')[0]) + ", " + df['sense2'].apply(lambda x: x.split('%')[0]),
#                        'target_text': df['Sentence']})
new_df = pd.DataFrame({'source_text': 'generate sentence: ' + context_df['Pun'] + ", " + context_df['Sense Key 1'].apply(lambda x: " ".join(x[0:2])) + ", " + context_df['Sense Key 2'].apply(lambda x: " ".join(x[0:2])),
                       'target_text': context_df['Sentence']})

train_df, test_df = train_test_split(new_df, test_size=0.2)
train_df.shape, test_df.shape

((1031, 2), (258, 2))

In [15]:
torch.cuda.empty_cache()

# assert torch.cuda.is_available()

# # Get the GPU device name.
# device_name = torch.cuda.get_device_name()
# n_gpu = torch.cuda.device_count()
# print(f"Found device: {device_name}, n_gpu: {n_gpu}")
# device = torch.device("cuda")
# model = SimpleT5()
# model.from_pretrained(model_type="t5", model_name="t5-base")
# model.train(train_df=train_df,
#             eval_df=test_df, 
#             source_max_token_len=128, 
#             target_max_token_len=50, 
#             batch_size=8, 
#             max_epochs=5, 
#             use_gpu=True
#            )
model = SimpleT5()
model = torch.load(PATH + "t5model-context.pt")

Found device: Tesla T4, n_gpu: 1


INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [17]:
# torch.save(model, 't5model-short-context.pt') 
# !cp t5model-short-context.pt /content/drive/MyDrive/CS678/AMBIPun 

In [38]:
model.device= torch.device("cpu")
def generateCandidateSentences(num_sentences, sense1, sense2, pun, model):
    candidates = []
    num_sentences = round(num_sentences / 3)
    for i in range(num_sentences):
        a = random.randint(0,len(sense1)-1)
        b = random.randint(0,len(sense2)-1)
        text="generate sentence: "+str(pun)+", "+str(sense1[a])+", "+str(sense2[b])
        candidates.append([model.predict(text)[0],text])
        text="generate sentence: "+str(sense1[a])+", "+str(pun)+", "+str(sense2[b])
        candidates.append([model.predict(text)[0],text])
        text="generate sentence: "+str(sense1[a])+", "+str(sense2[b])+", "+str(pun)
        candidates.append([model.predict(text)[0],text])
    return candidates

pun = "volume" # input pun word
# (rel_word1, rel_word2) = get_sense_rel_words(pun)
# sense1Cw = generateAllContextWords(rel_word1)
# sense2Cw = generateAllContextWords(rel_word2)
sense1Cw = ["Amount", "Size"] # input two context words from sense 1
sense2Cw = ["Amplifier", "Loud"] # input two context words from sense 2s
print(pun)
print(sense1Cw) 
print(sense2Cw)
sentences = generateCandidateSentences(10, sense1Cw, sense2Cw, pun, model)

print(sentences)

volume
['Amount', 'Size']
['Amplifier', 'Loud']
[['The inventor of a vacuum cleaner had a lot of volume. nan nan nan nan nan nan nan nan nan nan nan', 'generate sentence: volume, Size, Amplifier'], ['OLD PLANETS never die, they just lose their size. nan nan nan nan nan nan nan nan nan nan nan ', 'generate sentence: Size, volume, Amplifier'], ["''I've got a problem with the scale,'''said a scientist. nan nan nan nan nan ", 'generate sentence: Size, Amplifier, volume'], ['When he bought a guitar, there was no volume involved. nan nan nan nan nan nan nan nan nan nan nan ', 'generate sentence: volume, Amount, Loud'], ["The singer was only a minor actor's daughter, but she was an Amount. nan nan nan nan nan nan nan nan nan n", 'generate sentence: Amount, volume, Loud'], ["A man's face was full of amount. nan nan nan nan nan nan nan nan nan nan nan ", 'generate sentence: Amount, Loud, volume'], ['OLD PLAYERS never die they just drop out. nan nan nan nan nan nan nan nan nan nan nan nan', 'gen